# Fokker-Planck Equation III: Operator Splitting

This notebook investigates operator splitting methods for the Fokker Planck equation. Thus far in the notebook series, we have investigated the two terms in the FPE (Advection and Diffusion) separately, and in this notebook, we will investigate how these independent term-wise solutions can be used in combination to integrate the entire advection-diffusion equation, and discuss why it works, as well as the motivations and runtime savings afforded to the full implementation as a result.